In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
!cp /content/drive/MyDrive/2-folder/kaggle/df_utils.py /content/
import df_utils

In [8]:
def preprocess(df):
    df = df.copy()

    # def normalize_name(x):
    #     return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])

    def ticket_number(x):
        return x.split(" ")[-1]

    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])

    # df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)
    return df

In [28]:
df_train = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/titanic/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/titanic/test.csv')
# print(df_train.head(5))

df_train_x = preprocess(df_train)
df_train_x.Ticket_number = pd.to_numeric(df_train_x.Ticket_number, errors='coerce')
df_train_x = df_train_x.drop(['Ticket'], axis=1)
print(df_train_x.head(5))

def prepare_df(df, max_unique = 0, skip_cols=None):
  df_copy = df.copy()
  for column in df_copy:
    if column in skip_cols:
      continue
    print(column)
    column_type = df_copy[column].dtype
    check_nan = df_copy[column].isnull().values.any()
    # print(f'check_nan = {check_nan}')
    if column_type == 'object':
        # print(f'The column {column} contains string data')
        unique = df_copy[column].nunique()
        # print(f'unique = {unique}')
        if unique > max_unique:
          df_copy = df_copy.drop([column], axis=1)
          continue
        if check_nan:
          df_utils.fill_with_mode(column, df_copy)
        df_copy = df_utils.one_hot_encoding(df_copy, column)
    else:
        # print(f'The column {column} does not contain string data')
        if check_nan:
          df_utils.fill_with_mean(column, df_copy)
  return df_copy

df_train_x = prepare_df(df_train_x, 5, 'Survived')

# ----------------------------------------------------------------

y = df_train_x['Survived'].values
# print(y[0:5])
df_train_x = df_train_x.drop(['Survived'], axis=1)

# normalize dataframe
# df_train_x=(df_train_x-df_train_x.mean())/df_train_x.std()

X = df_train_x.values  #.astype(float)
# print(X[0:5])

X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.1, random_state=3)
print('Shape of X training set {}'.format(X_trainset.shape),'&',' Size of Y training set {}'.format(y_trainset.shape))
print('Shape of X test set {}'.format(X_testset.shape),'&','Size of y test set {}'.format(y_testset.shape))

print(f"X_trainset = {X_trainset.shape}")
print(f"X_testset = {X_testset.shape}")
print(f"y_trainset = {y_trainset.shape}")
print(f"y_testset = {y_testset.shape}")

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch     Fare Cabin Embarked  Ticket_number Ticket_item  
0      0   7.2500   NaN        S        21171.0         A/5  
1      0  71.2833   C85        C        17599.0          PC  
2      0   7.9250   NaN        S      3101282.0    STON/O2.  
3      0  53.1000  C123        S       113803.0        NONE  
4      0  

In [ ]:
# Dummy data (replace with real dataset)
# 100 samples, 10 features each
# X = torch.randn(100, 10)
# y = torch.randint(0, 3, (100,))  # 3 classes: 0, 1, 2
# print(X.shape)
# print(y.shape)

X_trainset_t=X_trainset_t.type(torch.float32)
X_testset_t=X_testset_t.type(torch.float32)
y_trainset_t=y_trainset_t.type(torch.float32)
y_testset_t=y_testset_t.type(torch.float32)

X = X_trainset_t
y = y_trainset_t

print(X_trainset_t.dtype)
print(X_trainset_t.shape)
print(y_trainset_t.dtype)
print(y_trainset_t.shape)

torch.float32
torch.Size([639, 10])
torch.float32
torch.Size([639])


In [ ]:
# X = torch.randn(100, 10)  # Features
# y = torch.randint(0, 2, (100,)).float()  # Binary labels (0 or 1)
# print(X.shape)
# print(X.dtype)
# print(y.shape)
# print(y.dtype)
X = X_trainset_t
y = y_trainset_t

# Dataset and DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=30, shuffle=True)

# Define the model
class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()  # Output probabilities

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Initialize the model, loss, and optimizer
input_size = 10
hidden_size = 32

model = BinaryClassifier(input_size, hidden_size)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    for batch in dataloader:
        inputs, labels = batch

        # Forward pass
        outputs = model(inputs).squeeze()  # Remove extra dimension
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/100], Loss: 0.7188
Epoch [2/100], Loss: 0.5303
Epoch [3/100], Loss: 0.6985
Epoch [4/100], Loss: 0.4456
Epoch [5/100], Loss: 0.6074
Epoch [6/100], Loss: 0.2632
Epoch [7/100], Loss: 0.5128
Epoch [8/100], Loss: 0.6000
Epoch [9/100], Loss: 0.2767
Epoch [10/100], Loss: 0.1942
Epoch [11/100], Loss: 0.5115
Epoch [12/100], Loss: 0.5976
Epoch [13/100], Loss: 0.7625
Epoch [14/100], Loss: 0.4983
Epoch [15/100], Loss: 0.3771
Epoch [16/100], Loss: 0.2547
Epoch [17/100], Loss: 0.2872
Epoch [18/100], Loss: 0.7483
Epoch [19/100], Loss: 0.4178
Epoch [20/100], Loss: 0.8189
Epoch [21/100], Loss: 0.3171
Epoch [22/100], Loss: 0.4610
Epoch [23/100], Loss: 0.6929
Epoch [24/100], Loss: 0.4097
Epoch [25/100], Loss: 0.8370
Epoch [26/100], Loss: 0.8206
Epoch [27/100], Loss: 0.3101
Epoch [28/100], Loss: 0.2841
Epoch [29/100], Loss: 0.5757
Epoch [30/100], Loss: 0.3653
Epoch [31/100], Loss: 0.4250
Epoch [32/100], Loss: 0.3185
Epoch [33/100], Loss: 0.6271
Epoch [34/100], Loss: 0.3745
Epoch [35/100], Loss: 0

In [ ]:
# Evaluation (Example)
model.eval()  # Set to evaluation mode
with torch.no_grad():
    # test_input = torch.randn(1, input_size)  # Example input
    print(X_testset_t.dtype)
    print(X_testset_t.shape)
    test_output = model(X_testset_t)
    test_output = test_output.squeeze()
    print(test_output.shape)
    print(test_output[:5])
    # predicted_class = 1 if test_output.item() > 0.5 else 0
    # print(f"Predicted class: {predicted_class}")
    test_output = test_output > 0.5
    print(test_output[:5])
    test_output = test_output.int()
    print(test_output[:5])

    y_testset_t = y_testset_t.int()
    print(y_testset_t.shape)
    print(y_testset_t[:5])

    test_result = test_output == y_testset
    print(test_result.shape)
    print(test_result[:5])

    test_result = test_result.int()
    print(test_result[:5])
    s = test_result.sum().item()
    l = test_result.shape[0]
    accuracy = round(s/l, 2)
    print(accuracy)

torch.float32
torch.Size([71, 10])
torch.Size([71])
tensor([0.0952, 0.1420, 0.9081, 0.8706, 0.0984])
tensor([False, False,  True,  True, False])
tensor([0, 0, 1, 1, 0], dtype=torch.int32)
torch.Size([71])
tensor([0, 1, 1, 1, 0], dtype=torch.int32)
torch.Size([71])
tensor([ True, False,  True,  True,  True])
tensor([1, 0, 1, 1, 1], dtype=torch.int32)
0.86


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/titanic/test.csv')
#print(df_train.head(10))

df_test_x = df_test.drop(['PassengerId', 'Name', 'Cabin'], axis=1)

# Categorical values to numeric
df_test_x = df_utils.one_hot_encoding(df_test_x, "Sex")
label_encoder = preprocessing.LabelEncoder()
df_test_x['Embarked']= label_encoder.fit_transform(df_test_x['Embarked'])

df_test_x = preprocess(df_test_x)
df_test_x = df_test_x.drop(['Ticket'], axis=1)
df_test_x['Ticket_item']= label_encoder.fit_transform(df_test_x['Ticket_item'])
df_test_x.Ticket_number = pd.to_numeric(df_test_x.Ticket_number, errors='coerce')

# Fix NaN
# print(df_test_x.isna().any())
df_utils.fill_with_mean("Age", df_test_x)
df_utils.fill_with_mean("Fare", df_test_x)

# print(f"df_test_x.shape[0] = {df_test_x.shape[0]}")
print(df_test_x.head(10))

# normalize dataframe
df_test_x=(df_test_x-df_test_x.mean())/df_test_x.std()
# print(df_test_x.head(10))
X_testset_t = torch.from_numpy(df_test_x.values)
X_testset_t=X_testset_t.type(torch.float32)
# print(X_testset_t.dtype)
# print(X_testset_t.shape)

model.eval()  # Set to evaluation mode
with torch.no_grad():
    # test_input = torch.randn(1, input_size)  # Example input
    print(X_testset_t.dtype)
    print(X_testset_t.shape)
    print(X_testset_t[5:])
    test_output = model(X_testset_t)
    test_output = test_output.squeeze()
    print(test_output.shape)
    print(test_output[:5])
    test_output = test_output > 0.5
    print(test_output[:5])
    test_output = test_output.int()
    print(test_output[:5])
    y_ = test_output.numpy()
    print(y_[:5])

predictions = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": y_
    })

def make_submission(predictions):
    path="/content/drive/MyDrive/2-folder/kaggle/titanic/submission.csv"
    predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

make_submission(predictions)

   Pclass   Age  SibSp  Parch     Fare  Embarked  Sex_female  Sex_male  \
0       3  34.5      0      0   7.8292         1         0.0       1.0   
1       3  47.0      1      0   7.0000         2         1.0       0.0   
2       2  62.0      0      0   9.6875         1         0.0       1.0   
3       3  27.0      0      0   8.6625         2         0.0       1.0   
4       3  22.0      1      1  12.2875         2         1.0       0.0   
5       3  14.0      0      0   9.2250         2         0.0       1.0   
6       3  30.0      0      0   7.6292         1         1.0       0.0   
7       2  26.0      1      1  29.0000         2         0.0       1.0   
8       3  18.0      0      0   7.2292         0         1.0       0.0   
9       3  21.0      2      0  24.1500         2         0.0       1.0   

   Ticket_number  Ticket_item  
0         330911           15  
1         363272           15  
2         240276           15  
3         315154           15  
4        3101298         